<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/adding_C1/C1/W4/assignment/C1_W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [1]:
#!pip install tensorflow==2.5.0

In [2]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [3]:
import tensorflow as tf
import os
import zipfile

#!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

#!wget --no-check-certificate \
#    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip \
#    -O /home/philipp/Downloads/happy-or-sad.zip

local_zip = '/home/philipp/Downloads/happy-or-sad.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

2021-08-31 14:27:30.605140: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


--2021-08-31 14:27:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.180.208, 172.217.16.112, 216.58.214.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.180.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2,5M) [application/zip]
Saving to: ‘/home/philipp/Downloads/happy-or-sad.zip’

/home/philipp/Downl 100%[===================>]   2,55M   909KB/s    in 2,9s    

2021-08-31 14:27:34 (909 KB/s) - ‘/home/philipp/Downloads/happy-or-sad.zip’ saved [2670333/2670333]



In [4]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999
    
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') >= DESIRED_ACCURACY):
                print("\nReached {}% accuracy so cancelling training!".format(DESIRED_ACCURACY))
                self.model.stop_training = True
            
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        # This is the first convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy']
                 )
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        './h-or-s/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=40,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(train_generator,
                        steps_per_epoch=2,  
                        epochs=15,
                        verbose=1
                       )
    
    return history.history['accuracy'][-1]

In [5]:
train_happy_sad_model()

2021-08-31 14:28:30.729536: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-31 14:28:30.730082: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-31 14:28:30.776982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 14:28:30.777439: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.7335GHz coreCount: 20 deviceMemorySize: 7.90GiB deviceMemoryBandwidth: 298.32GiB/s
2021-08-31 14:28:30.777457: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-08-31 14:28:30.778723: I tensorflow/stream_executor/platform/d

Found 80 images belonging to 2 classes.
Epoch 1/15


2021-08-31 14:28:31.606107: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-08-31 14:28:31.694974: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-08-31 14:28:32.127649: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-08-31 14:28:32.149019: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


2/2 [==============================] - 3s 19ms/step - loss: 3.2451 - accuracy: 0.4750
Epoch 2/15
2/2 [==============================] - 0s 34ms/step - loss: 1.1139 - accuracy: 0.5000
Epoch 3/15
2/2 [==============================] - 0s 40ms/step - loss: 0.6478 - accuracy: 0.8333
Epoch 4/15
2/2 [==============================] - 0s 38ms/step - loss: 0.5511 - accuracy: 0.7833
Epoch 5/15
2/2 [==============================] - 0s 34ms/step - loss: 0.3917 - accuracy: 0.9500
Epoch 6/15
2/2 [==============================] - 0s 38ms/step - loss: 1.3883 - accuracy: 0.4667
Epoch 7/15
2/2 [==============================] - 0s 44ms/step - loss: 0.4318 - accuracy: 0.9667
Epoch 8/15
2/2 [==============================] - 0s 39ms/step - loss: 0.3450 - accuracy: 0.7750
Epoch 9/15
2/2 [==============================] - 0s 38ms/step - loss: 0.2739 - accuracy: 0.9333
Epoch 10/15
2/2 [==============================] - 0s 34ms/step - loss: 0.2067 - accuracy: 0.9583
Epoch 11/15
2/2 [=======================

0.9624999761581421